# CS5760 — Homework 2

**Student Name:** Shaik Karishma  
**Student ID:** 700768890

Run cells top-to-bottom.

In [ ]:
import math
from collections import Counter


## Q5 — Confusion Matrix Metrics (Programming)

In [ ]:
# Confusion matrix: system rows x gold columns
labels = ["Cat", "Dog", "Rabbit"]
cm = [
    [5, 10, 5],   # predicted Cat
    [15, 20, 10], # predicted Dog
    [0, 15, 10],  # predicted Rabbit
]

def compute_metrics(cm, labels):
    n = len(labels)
    row_sums = [sum(cm[i]) for i in range(n)]  # predicted totals
    col_sums = [sum(cm[i][j] for i in range(n)) for j in range(n)]  # gold totals

    per_class = {}
    for k, lab in enumerate(labels):
        tp = cm[k][k]
        precision = tp / row_sums[k] if row_sums[k] else 0.0
        recall = tp / col_sums[k] if col_sums[k] else 0.0
        per_class[lab] = (precision, recall, tp, row_sums[k], col_sums[k])

    macro_p = sum(per_class[lab][0] for lab in labels) / n
    macro_r = sum(per_class[lab][1] for lab in labels) / n

    total_correct = sum(cm[i][i] for i in range(n))
    total = sum(sum(row) for row in cm)
    micro_p = total_correct / total
    micro_r = micro_p  # multiclass single-label

    return per_class, (macro_p, macro_r), (micro_p, micro_r)

per_class, (macro_p, macro_r), (micro_p, micro_r) = compute_metrics(cm, labels)

print("Per-class metrics:")
for lab in labels:
    p, r, tp, pred_total, gold_total = per_class[lab]
    print(f"  {lab}: TP={tp}, Pred={pred_total}, Gold={gold_total}, Precision={p:.4f}, Recall={r:.4f}")

print("\nMacro Precision:", round(macro_p, 4))
print("Macro Recall:", round(macro_r, 4))
print("Micro Precision:", round(micro_p, 4))
print("Micro Recall:", round(micro_r, 4))


## Part II Q1 — Bigram Language Model (MLE)

In [ ]:
# Training corpus (as given)
corpus_sents = [
    ["<s>", "I", "love", "NLP", "</s>"],
    ["<s>", "I", "love", "deep", "learning", "</s>"],
    ["<s>", "deep", "learning", "is", "fun", "</s>"],
]

# Unigram and bigram counts
unigram = Counter()
bigram = Counter()

for sent in corpus_sents:
    unigram.update(sent)
    bigram.update(zip(sent[:-1], sent[1:]))

def p_bigram_mle(h, w):
    denom = unigram[h]
    return (bigram[(h, w)] / denom) if denom else 0.0

def sentence_probability(tokens):
    prob = 1.0
    for h, w in zip(tokens[:-1], tokens[1:]):
        prob *= p_bigram_mle(h, w)
    return prob

S1 = ["<s>", "I", "love", "NLP", "</s>"]
S2 = ["<s>", "I", "love", "deep", "learning", "</s>"]

p1 = sentence_probability(S1)
p2 = sentence_probability(S2)

print("P(S1) =", p1)
print("P(S2) =", p2)

if p1 > p2:
    print("Preferred: S1 (<s> I love NLP </s>) because it has higher probability.")
elif p2 > p1:
    print("Preferred: S2 (<s> I love deep learning </s>) because it has higher probability.")
else:
    print("Both sentences have equal probability.")


## Part I Q3/Q4 

In [ ]:
# Bigram table from HW2 Part I Q3
counts = {
    "<s>": {"I": 2, "deep": 1},
    "I": {"love": 2},
    "love": {"NLP": 1, "deep": 1},
    "deep": {"learning": 2},
    "learning": {"</s>": 1, "is": 1},
    "NLP": {"</s>": 1},
    "is": {"fun": 1},
    "fun": {"</s>": 1},
    "ate": {"lunch": 6, "dinner": 3, "a": 2, "the": 1},
}

def total_after(h):
    return sum(counts.get(h, {}).values())

def mle(h, w):
    denom = total_after(h)
    return (counts.get(h, {}).get(w, 0) / denom) if denom else 0.0

def sent_prob(tokens):
    p = 1.0
    for h, w in zip(tokens[:-1], tokens[1:]):
        p *= mle(h, w)
    return p

S1 = ["<s>", "I", "love", "NLP", "</s>"]
S2 = ["<s>", "I", "love", "deep", "learning", "</s>"]
print("P(S1) =", sent_prob(S1))
print("P(S2) =", sent_prob(S2))
print("Preferred:", "S1" if sent_prob(S1) > sent_prob(S2) else "S2")

print("MLE P(noodle|ate) =", mle("ate", "noodle"))

# Add-1 smoothing (given |V|=10, total after ate=12)
V = 10
N_after_ate = 12
print("Add-1 P(noodle|ate) =", (0 + 1) / (N_after_ate + V))

# Backoff checks from Q4
print("P(cats|I,like) =", 1/2)
print("Backoff P(dogs|like) =", 1/3)
